# Load data into database 

Now we use our created csv file to create tables and fill the data into the tables.    
Additionally we will filter out exposes that are already existing. 

In [1]:
# Importing modules
from datetime import datetime
import pandas as pd
from pandas_profiling import ProfileReport
import hashlib
import numpy as np

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.mode.chained_assignment = None

DEBUG = True


In [2]:
# import paths for 
import sys
sys.path.insert(1, '../etl_oltp')
sys.path.insert(2, '../etl_staging')
sys.path.insert(3, '../data')


In [3]:
path_csv = "../data/immo.csv"

# read csv file
df = pd.read_csv(path_csv, dtype={'zip':'str'})
df.shape

(650, 157)

In [4]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 157 columns):
 #    Column                                   Dtype  
---   ------                                   -----  
 0    id                                       object 
 1    title                                    object 
 2    zip                                      object 
 3    buyingPrice                              float64
 4    rooms                                    float64
 5    squareMeter                              float64
 6    comission                                float64
 7    rentPricePerSqm                          float64
 8    pricePerSqm                              float64
 9    rentPrice                                float64
 10   rentPriceCurrent                         float64
 11   rentPriceCurrentPerSqm                   float64
 12   region                                   object 
 13   foreClosure                              bool   
 14   grossRet

In [5]:
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile

# Inspection
We will check what fields will be always available and where we have missing values.   

In [5]:
df.sample(1)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
186,6fac8084e86ea238aae95c1e7903f18d,Großzügige 3-Zimmer Wohnung mit Dachterrasse ...,24146,259000.0,3.0,91.0,2.98,8.0,2846.153846,728.0,NaN,NaN,Schleswig-Holstein,False,3.37,NaN,1979.0,ROOF_STOREY,MODERNIZED,2020.0,False,1.0,NaN,True,False,True,False,2021-02-22T15:11:36.000Z,False,False,116.666667,NaN,"[{'id': '07aa98bcccd98bdabea8c02676a78da0', 'o...",False,False,0,2021-02-22T17:17:09.898Z,0,NaN,-293.88,-13.616216,-293.88,True,DE,DEU,unknown,landuse,Kiel,NaN,Europe,Deutschland,de,European Union,24146.0,Schleswig-Holstein,SH,54.289597,10.17869,"Friedhof Elmschenhagen, 24146 Kiel, Deutschland",246794,-5,5,True,8.7,35972,67,0.0,25,25.0,16.664,Elmschenhagen,186550.0,2050.0,4.858537,Kiel,245700.0,2700,3.688889,NaN,222000.0,2439.56044,3.935135,NaN,24146.0,Kiel-Elmschenhagen,Elmschenhagen,24146,Kiel-Elmschenhagen,Elmschenhagen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

## Inspect locations table attributes

In [6]:
df_locations = df[['id',
    'zip','address.postcode','originalAddress.postcode', 'oAddress.is24.postcode',
    'region', 'address.state_code',
    'address.city','address.town','aggregations.location.name','oAddress.is24.location','originalAddress.location','oAddress.ebk.location', 'address.village',
    'address.city_district','oAddress.is24.district', 'originalAddress.district','address.borough','address.suburb',
    'address.county',
    'address.displayName', 'oAddress.ebk.locationQuery','originalAddress.locationQuery',
    'address.lat','address.lon']]
df_locations[:3]


,id,zip,address.postcode,originalAddress.postcode,oAddress.is24.postcode,region,address.state_code,address.city,address.town,aggregations.location.name,oAddress.is24.location,originalAddress.location,oAddress.ebk.location,address.village,address.city_district,oAddress.is24.district,originalAddress.district,address.borough,address.suburb,address.county,address.displayName,oAddress.ebk.locationQuery,originalAddress.locationQuery,address.lat,address.lon
0,f5b7641a8b8198cab2bc469d5c1b8913,69228,68229.0,69228,69228.0,Baden-Württemberg,BW,Mannheim,NaN,Mannheim,Mannheim,Mannheim,NaN,NaN,Friedrichsfeld,Friedrichsfeld,Friedrichsfeld,NaN,NaN,NaN,"68229 Mannheim, Deutschland",NaN,NaN,49.435667,8.572979
1,9572da5b09f5f6f09dcbe1c4ee8a0ef7,71083,71083.0,71083,NaN,Baden-Württemberg,BW,NaN,Herrenberg,Herrenberg,NaN,Herrenberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Landkreis Böblingen,"71083 Herrenberg, Deutschland",NaN,NaN,48.595557,8.867472
2,f0408586846aca702fdeb66647fe60b7,12049,12049.0,12049,12049.0,Berlin,BE,Berlin,NaN,Berlin,Berlin,Berlin,NaN,NaN,NaN,Neukölln (Ortsteil),Neukölln (Ortsteil),Neukölln,Neukölln,NaN,"Netzwerk behinderter Frauen Berlin e.V., Leine...",NaN,NaN,52.473055,13.426049


In [7]:
df_locations.isnull().sum()

id                                 0
zip                                0
address.postcode                  19
originalAddress.postcode           0
oAddress.is24.postcode           251
region                             0
address.state_code                 6
address.city                     278
address.town                     417
aggregations.location.name         9
oAddress.is24.location           251
originalAddress.location          53
oAddress.ebk.location            489
address.village                  564
address.city_district            480
oAddress.is24.district           416
originalAddress.district         405
address.borough                  573
address.suburb                   308
address.county                   339
address.displayName                0
oAddress.ebk.locationQuery       434
originalAddress.locationQuery    444
address.lat                        0
address.lon                        0
dtype: int64

region, zip and display name will always be given. 
City is not always given. Let's inspect if we get a good city balance.

In [8]:
# 'address.city','address.town','aggregations.location.name','oAddress.is24.location','originalAddress.location','oAddress.ebk.location', 
df_locations[
    (df_locations['address.city'].notnull()) | 
    (df_locations['address.town'].notnull()) | 
    (df_locations['aggregations.location.name'].notnull()) | 
    (df_locations['oAddress.is24.location'].notnull()) | 
    (df_locations['originalAddress.location'].notnull()) | 
    (df_locations['oAddress.ebk.location'].notnull())
].sample(3)[['address.city','address.town','aggregations.location.name','oAddress.is24.location','originalAddress.location','oAddress.ebk.location']]

,address.city,address.town,aggregations.location.name,oAddress.is24.location,originalAddress.location,oAddress.ebk.location
12,NaN,NaN,Birkweiler,NaN,birkweiler,birkweiler
392,Kreis Minden-Lübbecke,Minden,Minden,NaN,minden,minden
27,Lübeck,NaN,Lübeck,NaN,lübeck,lübeck


In [9]:
# 'address.city','address.town','aggregations.location.name','oAddress.is24.location','originalAddress.location','oAddress.ebk.location', 
df_locations[
    (df_locations['address.city'].notnull()) | 
    (df_locations['address.town'].notnull()) | 
    (df_locations['aggregations.location.name'].notnull()) | 
    (df_locations['oAddress.is24.location'].notnull()) | 
    (df_locations['originalAddress.location'].notnull()) | 
    (df_locations['oAddress.ebk.location'].notnull())
].sample(10)[['address.city','address.town','aggregations.location.name','oAddress.is24.location','originalAddress.location','oAddress.ebk.location']].isnull().sum()

address.city                  5
address.town                  5
aggregations.location.name    0
oAddress.is24.location        4
originalAddress.location      2
oAddress.ebk.location         9
dtype: int64

In [10]:
df_locations[['address.city','address.town','aggregations.location.name','oAddress.is24.location','originalAddress.location','oAddress.ebk.location']].isnull().sum()

address.city                  278
address.town                  417
aggregations.location.name      9
oAddress.is24.location        251
originalAddress.location       53
oAddress.ebk.location         489
dtype: int64

In [11]:
# 'address.city','address.town','aggregations.location.name','oAddress.is24.location','originalAddress.location','oAddress.ebk.location', 
df[
    (df_locations['address.city'].isnull()) & 
    (df_locations['address.town'].isnull()) & 
    (df_locations['aggregations.location.name'].isnull()) & 
    (df_locations['oAddress.is24.location'].isnull()) & 
    (df_locations['originalAddress.location'].isnull()) & 
    (df_locations['oAddress.ebk.location'].isnull())
]

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
198,b90c88833a8e8a9dbcefb478952075bc,Neubau von 9 Komfort-Eigentumswohnungen mit Te...,97461,185100.0,2.0,57.41,NaN,7.908030,3224.176973,454.0,NaN,NaN,Bayern,False,2.94,NaN,2021.0,GROUND_FLOOR,FIRST_TIME_USE,NaN,False,NaN,NaN,True,False,True,False,2021-02-22T15:05:00.271Z,False,False,120.194805,NaN,[],False,False,0,2021-02-22T15:05:00.915Z,0,NaN,-221.15,-14.337115,-221.15,False,DE,DEU,postcode,postcode,NaN,NaN,Europe,Deutschland,de,European Union,97461.0,Bayern,BY,50.138731,10.541062,"97461 Bayern, Deutschland",0,-5,5,False,0.0,0,17,0.0,0,0.0,16.664,NaN,NaN,NaN,NaN,NaN,200935.0,3500,3.154286,NaN,154000.0,2682.459502,3.537662,NaN,NaN,NaN,NaN,97461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bayern hofheim unterfr.,NaN,NaN,NaN,NaN,bayern hofheim unterfr.,97461.0,"97461 Bayern, Hofheim Unterfr.","97461 Bayern, Hofheim Unterfr.",NaN,NaN,NaN,NaN,NaN,Na

So we have exposes with only a state. Those are only 4 out of 600.   
1. We can drop them --> we will loose some exposes.   
2. We will identify them by zip. --> This is the preferred solution.   

In [12]:
df_locations[(df_locations['address.town'].notnull()) & (df_locations['aggregations.location.name'] != df_locations['address.town'])].head()

,id,zip,address.postcode,originalAddress.postcode,oAddress.is24.postcode,region,address.state_code,address.city,address.town,aggregations.location.name,oAddress.is24.location,originalAddress.location,oAddress.ebk.location,address.village,address.city_district,oAddress.is24.district,originalAddress.district,address.borough,address.suburb,address.county,address.displayName,oAddress.ebk.locationQuery,originalAddress.locationQuery,address.lat,address.lon


In [13]:
df_locations[(df_locations['address.city'].isnull()) & (df_locations['address.town'].notnull())].shape

(194, 25)

In [14]:
df[df_locations['address.city'].isnull()][:1]


,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
1,9572da5b09f5f6f09dcbe1c4ee8a0ef7,Penthouse mit Aussicht in urbaner Lage. Zeit z...,71083,535000.0,3.0,96.88,4.44,12.933526,5522.295623,1253.0,NaN,NaN,Baden-Württemberg,False,2.81,NaN,2021.0,PENTHOUSE,FIRST_TIME_USE,NaN,False,2.0,4.0,False,False,True,False,2021-02-22T17:10:36.860Z,False,False,110.309278,NaN,"[{'id': '86919a5116d0de3adc8cc2d5e5f06b59', 'o...",False,False,0,2021-02-22T17:10:38.213Z,0,NaN,-769.3,-17.255327,-769.3,True,DE,DEU,place,city,NaN,NaN,Europe,Deutschland,de,European Union,71083.0,Baden-Württemberg,BW,48.595557,8.867472,"71083 Herrenberg, Deutschland",31738,-5,5,False,4.1,0,52,18.658537,0,16.66,16.664,NaN,NaN,NaN,NaN,Herrenberg,324548.0,3350,3.797015,NaN,485000.0,5006.193229,3.100206,NaN,NaN,NaN,NaN,71083,Herrenberg,NaN,Landkreis Böblingen,Vereinbarte Verwaltungsgemeinschaft der Stadt ...,Herrenberg,NaN,71083.0,Herrenberg,NaN,NaN,NaN,NaN

In [15]:
df_locations['region'].unique()


array(['Baden-Württemberg', 'Berlin', 'Nordrhein-Westfalen', 'Hessen',
       'Bayern', 'Rheinland-Pfalz', 'Brandenburg', 'Niedersachsen',
       'Sachsen', 'Saarland', 'Schleswig-Holstein', 'Hamburg',
       'Sachsen-Anhalt', 'Freie Hansestadt Bremen',
       'Mecklenburg-Vorpommern', 'Thüringen'], dtype=object)

We will always have all states given. 

How to create a table that has the city information given before we drop the duplicate zips???? TODO!!!

In [16]:
df_temp = df.drop_duplicates('zip', keep='last')
df_temp[:2]

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
0,f5b7641a8b8198cab2bc469d5c1b8913,"Attraktive 1,5-Zimmer-Wohnung zwischen Mannhei...",69228,129000.0,1.0,48.00,4.44,9.791667,2687.5,470.0,NaN,NaN,Baden-Württemberg,False,4.37,NaN,1984.0,APARTMENT,WELL_KEPT,NaN,False,NaN,1.0,True,False,True,False,2021-02-22T17:12:55.000Z,False,False,100.781250,NaN,"[{'id': '588ca284e7e9d0cfd67ac36c75492073', 'o...",False,False,0,2021-02-22T17:14:40.252Z,0,NaN,-76.58,-7.124031,-76.58,True,DE,DEU,place,neighbourhood,Mannheim,Friedrichsfeld,Europe,Deutschland,de,European Union,68229.0,Baden-Württemberg,BW,49.435667,8.572979,"68229 Mannheim, Deutschland",310658,-5,5,True,7.8,19417,76,9.807692,25,25.0,16.664,Friedrichsfeld,127200.0,2650.0,4.030189,Mannheim,148800.0,3100,3.870968,NaN,128000.0,2666.666667,4.406250,NaN,69228.0,Mannheim,Friedrichsfeld,69228,Mannheim,Friedrichsfeld,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## Inspect district


In [17]:
'address.city_district','oAddress.is24.district', 'originalAddress.district','address.borough','address.suburb'
# 'address.city','address.town','aggregations.location.name','oAddress.is24.location','originalAddress.location','oAddress.ebk.location', 
df_locations[
    (df_locations['address.city_district'].notnull()) | 
    (df_locations['oAddress.is24.district'].notnull()) | 
    (df_locations['originalAddress.district'].notnull()) | 
    (df_locations['address.borough'].notnull()) | 
    (df_locations['address.suburb'].notnull())
][['address.city_district','oAddress.is24.district', 'originalAddress.district','address.borough','address.suburb']].isnull().sum()

address.city_district       248
oAddress.is24.district      184
originalAddress.district    173
address.borough             341
address.suburb               76
dtype: int64

### Inspect commission

In [18]:
df['comission'].sort_values(ascending=False)[:5]

358    19.00
351    19.00
226    19.00
150    10.31
188     9.30
Name: comission, dtype: float64

## Inspect unique own id

In [19]:
len((df['title'] + df['zip'] + df['squareMeter'].astype(str)).unique())
len((df['title'].unique())),len((df['title'] + df['zip']).unique())

(583, 588)

In [20]:
df_uniques = df.drop_duplicates('id')
df_uniques.shape

(600, 157)

In [21]:
df_temp_unique = df_uniques[['id','title']]
df_temp_unique['unique_id'] = (df_uniques['title'] + df_uniques['zip'] + df_uniques['squareMeter'].astype(str))
df_uniques[df_temp_unique['unique_id'].duplicated(keep=False)]

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
129,a9f1966542a947a7ddac324dd01cb692,Direkt an der Südpfalz-Therme: 1-Zi.-Apartment...,76887,125000.0,1.0,54.05,3.57,7.622572,2312.673451,412.0,NaN,NaN,Rheinland-Pfalz,False,3.96,NaN,1985.0,APARTMENT,WELL_KEPT,2017.0,False,NaN,NaN,True,False,True,False,2021-02-22T15:51:29.736Z,False,False,115.740741,NaN,[],False,False,0,2021-02-22T15:51:29.832Z,0,NaN,-99.70,-9.571200,-99.70,False,DE,DEU,postcode,postcode,NaN,NaN,Europe,Deutschland,de,European Union,76887.0,Rheinland-Pfalz,RP,49.103257,7.999523,"76887 Bad Bergzabern, Deutschland",8285,-5,5,False,4.7,0,33,16.276596,0,0.0,16.664,NaN,NaN,NaN,NaN,Bad Bergzabern,108100.0,2000,4.440000,NaN,108000.0,1998.149861,4.577778,NaN,NaN,NaN,NaN,76887,bad bergzabern,NaN,Landkreis Südliche Weinstraße,Bad Bergzabern,Bad Bergzabern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76887.0,"76887 Rheinland-Pfalz, Bad Ber

Optimal combination of attributes in order to build a primary key is (zip,title,squaremeter)

In [22]:
len(df.id.unique())

600

In [23]:
df[['id','title','zip','squareMeter']].sample(3)

,id,title,zip,squareMeter
28,b75639cab589f49eeacd43bf6daaec1b,Vermietete Zweizimmerwohnung in zentraler Lage...,45879,55.0
592,eec3c1ddb25a784382cfe02406094ec5,Kapitalanlage im schönen Fellbach,70736,50.0
560,0c8465b235d4c071e7a7ca06b07e1ba0,"prov.-frei, 5Zi, Aussicht, 2 OG, Reinsburg, zu...",70197,136.0


### Inspect Expose location
### Street

In [24]:
df[['oAddress.is24.street', 'address.road', 'originalAddress.street']].head()

,oAddress.is24.street,address.road,originalAddress.street
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,Leinestraße 51,Leinestraße,Leinestraße 51
3,NaN,NaN,NaN
4,NaN,NaN,NaN


### Display name

In [25]:
df[['address.displayName']].head(3)

,address.displayName
0,"68229 Mannheim, Deutschland"
1,"71083 Herrenberg, Deutschland"
2,"Netzwerk behinderter Frauen Berlin e.V., Leine..."


In [26]:
df.sample(3)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
132,ff96205db8bf82d9d967645c65db3689,Findorff / Helle 3-Zimmer-Wohnung in begehrter...,28215,169000.0,3.0,66.0,3.00,8.606061,2560.606061,568.0,NaN,NaN,Freie Hansestadt Bremen,False,4.03,NaN,1952.0,APARTMENT,WELL_KEPT,NaN,False,1.0,NaN,True,False,True,False,2021-02-22T15:50:07.000Z,False,False,98.255814,194.0,"[{'id': 'b01ef35be268bc244a1e1e983370e315', 'o...",False,False,0,2021-02-22T15:52:08.460Z,0,NaN,-123.72,-8.784615,-123.72,True,DE,DEU,road,road,Bremen,Stadtbezirk Bremen-West,Europe,Deutschland,de,European Union,28215.0,Freie Hansestadt Bremen,NaN,53.095526,8.805172,"Regensburger Straße, 28215 Bremen, Deutschland",567559,-5,5,True,11.0,32971,67,0.000000,25,25.0,16.664,Stadtbezirk Bremen-West,165000.0,2500.0,4.128000,Bremen,165000.0,2500,4.128000,NaN,172000.0,2606.060606,3.962791,NaN,28215.0,Bremen,Regensburger Straße,28215,Bremen,Regensburger Straße,NaN,

# Create tables

In [27]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

DB_HOST = os.getenv('DB_HOST')
DB_NAME  = os.getenv('DB_NAME')
DB_USER  = os.getenv('DB_USER')
DB_PASSWORD  = os.getenv('DB_PASSWORD')
DB_SCHEMA = os.getenv('DB_SCHEMA')

connection_parameter = {
    "host"      : DB_HOST,
    "database"  : DB_NAME,
    "user"      : DB_USER,
    "password"  : DB_PASSWORD
}

def connect(connection_parameter):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**connection_parameter)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("Connection successful")
    return conn

def execute_query(connection, query):
    try:
        cursor = connection.cursor()

        try:
            cursor.execute(query)
        except psycopg2.IntegrityError:
            connection.rollback()
        else:
            connection.commit()

        cursor.close() 
    except (Exception, psycopg2.DatabaseError) as error:
        print('ERROR while query:', error)
        sys.exit(1)
    print("Query DONE")

    # cursor = connection.cursor()
    # try:
    #     print("Query execute")
    #     # connect to the PostgreSQL server
    #     cursor.execute(query)
    #     connection.commit()
    #     cursor.close()
    # except (Exception, psycopg2.DatabaseError) as error:
    #     print(error)
    #     cursor.rollback()
    #     cursor.close()
    #     sys.exit(1)
    # print("Query DONE")
    # with connection.cursor() as cursor:
    #     cursor.execute(query)

    # with closing(connection.cursor()) as cursor:
    # TODO delete old way because cursor does not close if exception thrown
    # cursor = connection.cursor()
    # cursor.execute(query)
    # connection.commit()
    # cursor.close()

def execute_query_many(connection, query, values):
    try:
        cursor = connection.cursor()

        try:
            # Create a list of tupples from the dataframe values
            tuples = [tuple(x) for x in values[:400]]
            # Comma-separated dataframe columns
            # cols = ','.join(list(df.columns)) # cols are given in query
            # SQL quert to execute
            placeholders = "(" + "%s,"*len(values[0])
            placeholders = placeholders[:-1] + ")"
            placeholders
            values = [cursor.mogrify(placeholders, tup).decode('utf8') for tup in tuples]
            
            # append values to query
            query = query + ",".join(values)
            print(f"Query: {query}")

            cursor.execute(query, values)
            #cursor.executemany(query, values)
        except psycopg2.IntegrityError:
            connection.rollback()
        else:
            connection.commit()

        cursor.close() 
    except (Exception, psycopg2.DatabaseError) as error:
        print('ERROR while many query:', error)
        sys.exit(1)
    print("Many query DONE")
    # cursor = connection.cursor()
    # try:
    #     print("Many query execute")
    #     # connect to the PostgreSQL server
    #     cursor.executemany(query,values)
    #     connection.commit()
    #     cursor.close()
    # except (Exception, psycopg2.DatabaseError) as error:
    #     print(error)
    #     cursor.rollback()
    #     cursor.close()
    #     sys.exit(1)
    # print("Many query DONE")
    # with connection.cursor() as cursor:
    #     cursor.executemany(query,values)

    # with closing(connection.cursor()) as cursor:
    # cursor = connection.cursor()
    # cursor.executemany(query,values)
    # connection.commit()
    # cursor.close()

def postgresql_to_dataframe(connection, select_query):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    # cursor = conn.cursor()
    # try:
    #     cursor.execute(select_query)
    # except (Exception, psycopg2.DatabaseError) as error:
    #     print("Error: %s" % error)
    #     cursor.close()
    #     return 1
    
    # # Naturally we get a list of tupples
    # tupples = cursor.fetchall()
    # cursor.close()
    print("Read from table")
    df = pd.read_sql_query(select_query,con=connection)
    
    # We just need to turn it into a pandas dataframe
    # df = pd.DataFrame(tupples, columns=column_names)
    return df

def get_hash(string_to_hash):
    try:
        cleaned_string = string_to_hash.upper().replace(" ", "").encode("utf8")
        md5hash = hashlib.md5(cleaned_string).hexdigest()
        return md5hash
    except:
        return None

In [28]:
# # (%s,%s,%s)
# placeholders = "(" + "%s,"*10
# placeholders = placeholders[:-1] + ")"
# placeholders

In [29]:
# values = [[1,2,3],[4,5,6]]
# tuples = [tuple(x) for x in values]
# print(tuples)
# cursor = connection.cursor()
# values_mogrify = [cursor.mogrify("(%s)", tup).decode('utf8') for tup in tuples]
# print(values_mogrify)
# query = ",".join(values_mogrify)
# query

In [30]:
connection = connect(connection_parameter)
connection

Connecting to the PostgreSQL database...
Connection successful


<connection object at 0x131ae6c10; dsn: 'user=postgres password=xxx dbname=property_listings host=liveai.cd94ktqj8jv1.eu-central-1.rds.amazonaws.com', closed: 0>

## Create table locations

In [31]:
query_drop_table_if_exists = "DROP TABLE IF EXISTS {table_name}"
query_create_table_locations = """ 
    CREATE TABLE {table_name}(
        postal_code VARCHAR(5) PRIMARY KEY,
        state VARCHAR(200) NULL,
        city VARCHAR(200) NOT NULL,
        longitude DECIMAL NULL,
        latitude DECIMAL NULL,
        slug VARCHAR(50) NULL,
        rent_price_index_lower_30sqm DECIMAL NULL,
        rent_price_index_lower_60sqm DECIMAL NULL,
        rent_price_index_equal_above_60sqm DECIMAL NULL,
        notary_rate DECIMAL NULL,
        real_estate_transfer_tax DECIMAL NULL,
        land_register_tax DECIMAL NULL,
        creation_date DATE NOT NULL DEFAULT CURRENT_DATE,
        updated_date DATE NOT NULL DEFAULT CURRENT_DATE	
);
"""

In [32]:
def drop_table(connection, table_name):
    execute_query(connection, query_drop_table_if_exists.format(table_name=table_name))

In [33]:
# TODO bring back
# drop_table(connection, f"{DB_SCHEMA}.locations")
# execute_query(connection, query_create_table_locations.format(table_name=f"{DB_SCHEMA}.locations"))

## Fill table locations
We have to get the distinct values and fill it into a dataframe
We will always have a zip. And in most cases a city. 

        expose_id   city    town    real_city
row1    1           null    Berlin   None
row2    1           Berlin  Berlin   None
row3    1           null    Berlin   None

row     1           None    None     Berlin

In [34]:
def get_new_locations(df, existing_zips):
    print("Create locations values")
    before  = datetime.now()                         
    # drop duplicate locations (by zip)
    df_temp = df.drop_duplicates('zip', keep='first') # Problem: drop city information
    # create locations datframe
    columns_locations = ["postal_code", "state", "city", "longitude", "latitude", "slug", "rent_price_index_lower_30sqm", "rent_price_index_lower_60sqm", "rent_price_index_equal_above_60sqm", "notary_rate", "real_estate_transfer_tax", "land_register_tax"] 
    df_locations_table = pd.DataFrame(columns=columns_locations)
    df_locations_table['postal_code'] = df_temp['zip']
    df_locations_table['state'] = df_temp['region']
    df_locations_table['city'] = df_temp['address.city']
    df_locations_table['city'] = df_locations_table['city'].fillna(df_locations['address.town']).fillna(df_locations['aggregations.location.name']).fillna(df_locations['oAddress.is24.location']).fillna(df_locations['originalAddress.location']).fillna(df_locations['oAddress.ebk.location'])
    df_locations_table['longitude'] = None
    df_locations_table['latitude'] = None
    df_locations_table['slug'] = None
    df_locations_table['rent_price_index_lower_30sqm'] = None
    df_locations_table['rent_price_index_lower_60sqm'] = None
    df_locations_table['rent_price_index_equal_above_60sqm'] = None
    df_locations_table['notary_rate'] = None
    df_locations_table['real_estate_transfer_tax'] = None
    df_locations_table['land_register_tax'] = None
    result = df_locations_table[~df_locations_table['postal_code'].isin(existing_postal_codes)]
    duration = (datetime.now() - before).total_seconds()                        
    print(f"Locations table created in {duration} seconds.")
    return result

# existing_postal_codes = [69228,12049,52074]
existing_postal_codes = []
df_locations_new = get_new_locations(df,existing_postal_codes)
df_locations_new.head(2)

Create locations values
Locations table created in 0.01321 seconds.


,postal_code,state,city,longitude,latitude,slug,rent_price_index_lower_30sqm,rent_price_index_lower_60sqm,rent_price_index_equal_above_60sqm,notary_rate,real_estate_transfer_tax,land_register_tax
0,69228,Baden-Württemberg,Mannheim,None,None,None,None,None,None,None,None,None
1,71083,Baden-Württemberg,Herrenberg,None,None,None,None,None,None,None,None,None


In [35]:
# query_insert_locations = """
#     INSERT INTO {table_name}(postal_code, state, city, longitude, latitude, slug, rent_price_index_lower_30sqm, rent_price_index_lower_60sqm, rent_price_index_equal_above_60sqm, notary_rate, real_estate_transfer_tax, land_register_tax)
#     VALUES {values};
# """
# query_insert_locations = """
#     INSERT INTO {table_name}(postal_code, state, city)
#     VALUES {values};
# """

# query_insert_locations = """
#     INSERT INTO live_ai_v2.locations (postal_code, state, city, longitude, latitude, slug, rent_price_index_lower_30sqm, rent_price_index_lower_60sqm, rent_price_index_equal_above_60sqm, notary_rate, real_estate_transfer_tax, land_register_tax)
#     VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
# """

query_insert_locations = """
    INSERT INTO live_ai_v2.locations (postal_code, state, city, longitude, latitude, slug, rent_price_index_lower_30sqm, rent_price_index_lower_60sqm, rent_price_index_equal_above_60sqm, notary_rate, real_estate_transfer_tax, land_register_tax)
    VALUES 
"""

def insert_locations(df):
    print("Insert locations")
    before  = datetime.now()                        
    execute_query_many(connection, query_insert_locations, df.values.tolist())
    duration = (datetime.now() - before).total_seconds()                        
    print(f"{df.shape[0]} locations inserted in {duration} seconds.")

# df.to_sql(replace) <-- storing data in ram 

# TODO bring back location insert
insert_locations(df_locations_new)

Insert locations
Query: 
    INSERT INTO live_ai_v2.locations (postal_code, state, city, longitude, latitude, slug, rent_price_index_lower_30sqm, rent_price_index_lower_60sqm, rent_price_index_equal_above_60sqm, notary_rate, real_estate_transfer_tax, land_register_tax)
    VALUES 
('69228','Baden-Württemberg','Mannheim',NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL),('71083','Baden-Württemberg','Herrenberg',NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL),('12049','Berlin','Berlin',NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL),('52074','Nordrhein-Westfalen','Aachen',NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL),('60323','Hessen','Frankfurt am Main',NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL),('93047','Bayern','Regensburg',NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL),('67071','Rheinland-Pfalz','Ludwigshafen am Rhein',NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL),('12169','Berlin','Berlin',NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL),('15526','Brandenburg','Bad Saarow',NULL,NULL,NULL,NUL

What is the best way (time) to insert data?    
1. executemany from PSQL   
2. df.to_sql <-- storing data in ram. Very memory intensive
3. ??? 

 How to insert values but not as duplicates. And also not overwrite existing rows.      
 1. sql query INSERT INTO --> this creates error because of duplicate primary key --> use try catch   
 2. 2 dataframes. --> 1 series for existing column of zips used as lookup for new created table (~isin())    
 3. anti join --> a join that adds only values that are not exist in the table.
  

In [36]:
df_locations.head(1)

,id,zip,address.postcode,originalAddress.postcode,oAddress.is24.postcode,region,address.state_code,address.city,address.town,aggregations.location.name,oAddress.is24.location,originalAddress.location,oAddress.ebk.location,address.village,address.city_district,oAddress.is24.district,originalAddress.district,address.borough,address.suburb,address.county,address.displayName,oAddress.ebk.locationQuery,originalAddress.locationQuery,address.lat,address.lon
0,f5b7641a8b8198cab2bc469d5c1b8913,69228,68229.0,69228,69228.0,Baden-Württemberg,BW,Mannheim,NaN,Mannheim,Mannheim,Mannheim,NaN,NaN,Friedrichsfeld,Friedrichsfeld,Friedrichsfeld,NaN,NaN,NaN,"68229 Mannheim, Deutschland",NaN,NaN,49.435667,8.572979


In [37]:
df.head(1)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
0,f5b7641a8b8198cab2bc469d5c1b8913,"Attraktive 1,5-Zimmer-Wohnung zwischen Mannhei...",69228,129000.0,1.0,48.0,4.44,9.791667,2687.5,470.0,NaN,NaN,Baden-Württemberg,False,4.37,NaN,1984.0,APARTMENT,WELL_KEPT,NaN,False,NaN,1.0,True,False,True,False,2021-02-22T17:12:55.000Z,False,False,100.78125,NaN,"[{'id': '588ca284e7e9d0cfd67ac36c75492073', 'o...",False,False,0,2021-02-22T17:14:40.252Z,0,NaN,-76.58,-7.124031,-76.58,True,DE,DEU,place,neighbourhood,Mannheim,Friedrichsfeld,Europe,Deutschland,de,European Union,68229.0,Baden-Württemberg,BW,49.435667,8.572979,"68229 Mannheim, Deutschland",310658,-5,5,True,7.8,19417,76,9.807692,25,25.0,16.664,Friedrichsfeld,127200.0,2650.0,4.030189,Mannheim,148800.0,3100,3.870968,NaN,128000.0,2666.666667,4.40625,NaN,69228.0,Mannheim,Friedrichsfeld,69228,Mannheim,Friedrichsfeld,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

## Create table source

In [38]:
len(df['platforms.url'].unique())

633

In [39]:
df.sample(1)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
239,048944ae1cc32a261989f1a01c6e6e8f,"Etagenwohnung in 04539 Groitzsch, Graf-Wiprech...",04539,45000.0,2.0,62.0,NaN,5.177419,725.806452,321.0,NaN,NaN,Sachsen,True,8.56,NaN,1930.0,APARTMENT,NO_INFORMATION,NaN,False,NaN,NaN,False,False,True,False,2021-02-22T14:49:43.385Z,False,False,102.272727,NaN,[],False,False,0,2021-02-22T17:03:08.571Z,0,NaN,83.23,22.195556,83.23,False,DE,DEU,postcode,postcode,NaN,NaN,Europe,Deutschland,de,European Union,4539.0,Sachsen,SN,51.144704,12.284832,"04539 Groitzsch, Deutschland",7586,-10,-5,False,6.0,0,25,12.75,0,0.0,12.498,NaN,NaN,NaN,NaN,Groitzsch,40300.0,650,9.230769,NaN,44000.0,709.677419,8.754545,NaN,NaN,NaN,NaN,4539,sachsen groitzsch,NaN,Landkreis Leipzig,NaN,Groitzsch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4539.0,"04539 Sachsen, Groitzsch","04539 Sachsen, Groitzsch",sachsen groitzsch,NaN,NaN,NaN,NaN,NaN,

In [40]:
def get_expose_id_hash(row): 
    return str(hash(row['zip'] + row['title'] + str(row['squareMeter'])))

def get_expose_id(df):
    return df.apply(lambda row: str(get_expose_id_hash(row)), axis = 1)

def get_source_id(df):
    return df.apply(lambda row: str(hash(get_expose_id_hash(row) + str(row['platforms.url']))), axis = 1)

def get_price_history_id(df):
    return df.apply(lambda row: str(hash(get_expose_id_hash(row) + str(row['buyingPriceHistory.platformName']))), axis = 1)

In [41]:
query_create_table_sources = """ 
    CREATE TABLE {table_name}(
        id VARCHAR(30) PRIMARY KEY,
        expose_id VARCHAR(30) NOT NULL,
        is_active BOOLEAN NOT NULL,
        source_name VARCHAR(200) NOT NULL,
        source_id VARCHAR(100) NULL,
        url VARCHAR(300) NOT NULL,
        creation_date DATE NOT NULL DEFAULT CURRENT_DATE,
        valid_until DATE NOT NULL DEFAULT '9999-12-31'
);
"""
        # CONSTRAINT fk_exposes FOREIGN KEY(expose_id) REFERENCES {table_name_foreign}(id) ON DELETE NO ACTION
        # CONSTRAINT fk_exposes FOREIGN KEY(expose_id) REFERENCES {table_name_foreign}(id) ON DELETE NO ACTION
query_create_table_price_history = """ 
    CREATE TABLE {table_name}(
        id VARCHAR(30) PRIMARY KEY,
        expose_id VARCHAR(30) NOT NULL,
        purchase_price DECIMAL NULL,
        source_name VARCHAR(50) NULL,
        creation_date DATE NOT NULL DEFAULT CURRENT_DATE
);
"""
# change not null values (price_id,source_id, postal_code, title)
query_create_table_exposes = """ 
    CREATE TABLE {table_name}(
        id VARCHAR(30) PRIMARY KEY,
        postal_code VARCHAR(5) NOT NULL,
        price_id INT NULL, 
        CONSTRAINT fk_locations FOREIGN KEY(postal_code) REFERENCES {table_name_foreign_location}(postal_code)
);
"""
# query_create_table_exposes = """ 
#     CREATE TABLE {table_name}(
#         id VARCHAR(20) PRIMARY KEY,
#         postal_code VARCHAR(5) NULL,
#         price_id INT NULL, 
#         source_id INT NULL,
#         title VARCHAR(200) NULL,
#         purchase_price DECIMAL NULL,
#         district VARCHAR(200) NULL,
#         street_number VARCHAR(5) NULL,
#         street VARCHAR(5) NULL,
#         address_display_name VARCHAR(400),
#         is_active BOOLEAN DEFAULT TRUE,
#         property_type VARCHAR(10) NULL, 
#         longitude DECIMAL NULL,
#         latitude DECIMAL NULL,
#         has_vaccancy BOOLEAN NULL,
#         is_foreclosure BOOLEAN NULL,
#         purchase_price_per_sqm DECIMAL NULL,
#         commission_rate DECIMAL NULL,
#         commission_total DECIMAL NULL,
#         management_cost_given DECIMAL NULL,
#         management_cost_estimated DECIMAL NULL,
#         rent_price_per_sqm_given DECIMAL NULL,
#         rent_price_per_sqm_estimated DECIMAL NULL,
#         rent_price_total_estimated DECIMAL NULL,
#         rent_price_total_given DECIMAL NULL,
#         city_slug VARCHAR(200) NULL,
#         condition VARCHAR(20) NULL,
#         construction_year INT NULL,
#         floors DECIMAL NULL,
#         at_floor DECIMAL NULL,
#         living_space DECIMAL NULL,
#         rooms VARCHAR(10) NULL,
#         refurbishment_maintenance_reserve DECIMAL NULL,
#         has_lift BOOLEAN NULL,
#         has_balcony BOOLEAN NULL,
#         is_private_offer BOOLEAN NULL,
#         has_garden BOOLEAN NULL,
#         image_captions TEXT NULL,
#         image_urls TEXT NULL,
#         cash_flow_gross DECIMAL NULL,
#         cash_flow_net DECIMAL NULL,
#         roi_equity DECIMAL NULL,
#         mortgage_monthly DECIMAL NULL,
#         mortgage_yearly DECIMAL NULL,
#         mortgage_total DECIMAL NULL, 
#         interest_monthly DECIMAL NULL,
#         interest_yearly DECIMAL NULL,
#         interest_total DECIMAL NULL,
#         personal_tax DECIMAL NULL, 
#         loan DECIMAL NULL,
#         private_maintenance_reserve DECIMAL NULL,
#         rent_price_index DECIMAL NULL,
#         notary_rate DECIMAL NULL,
#         real_estate_transfer_tax DECIMAL NULL,
#         land_register_tax DECIMAL NULL,
#         created_at DATE NULL DEFAULT CURRENT_DATE,
#         valid_until DATE NULL DEFAULT '9999-12-31'
# );
# """


In [42]:
# Create table
drop_table(connection, f"{DB_SCHEMA}.sources")
drop_table(connection, f"{DB_SCHEMA}.price_history")
drop_table(connection, f"{DB_SCHEMA}.exposes")
execute_query(connection, query_create_table_sources.format(table_name=f"{DB_SCHEMA}.sources",table_name_foreign=f"{DB_SCHEMA}.exposes"))
execute_query(connection, query_create_table_price_history.format(table_name=f"{DB_SCHEMA}.price_history",table_name_foreign=f"{DB_SCHEMA}.exposes"))
execute_query(connection, query_create_table_exposes.format(table_name=f"{DB_SCHEMA}.exposes",table_name_foreign_location=f"{DB_SCHEMA}.locations"))

Query DONE
Query DONE
Query DONE
Query DONE
Query DONE
Query DONE


In [43]:
def get_new_sources(df):
    print("Create source values")
    before  = datetime.now()                         
    # create locations datframe
    columns = [
        "id",
        "expose_id",
        "is_active",
        "source_name",
        "source_id",
        "url",
        "creation_date",
        'valid_until'
    ] 
    df_source_table = pd.DataFrame(columns=columns)
    df_source_table['id'] = get_source_id(df)
    df_source_table['expose_id'] = get_expose_id(df)
    df_source_table['is_active'] = df['platforms.active'] # is always True or False
    df_source_table['source_name'] = df['platforms.name']
    df_source_table['source_id'] = df['platforms.id']
    df_source_table['url'] = df['platforms.url']
    df_source_table['creation_date'] = df['platforms.creationDate']
    df_source_table['valid_until'] = np.where(df['platforms.deactivationDate'].notnull(),df['platforms.deactivationDate'],'9999-12-31')

    duration = (datetime.now() - before).total_seconds()                        
    print(f"Source table created in {duration} seconds.")
    return df_source_table

df_sources_new = get_new_sources(df)
df_sources_new.head(3)

Create source values
Source table created in 0.043664 seconds.


,id,expose_id,is_active,source_name,source_id,url,creation_date,valid_until
0,3884518475819788511,6400432849230923943,True,is24,126782974,https://www.immobilienscout24.de/expose/126782974,2021-02-22T17:12:55.000Z,9999-12-31
1,-7862558759484577145,3402233425913400653,True,immobilio,immobilioBMYJZS,https://www.immobilo.de/immobilien/penthouse-m...,2021-02-22T17:10:36.860Z,9999-12-31
2,-1339679137021876320,5910895991693760749,True,is24,126782931,https://www.immobilienscout24.de/expose/126782931,2021-02-22T17:08:54.000Z,9999-12-31


In [44]:
query_insert_sources = """
    INSERT INTO live_ai_v2.sources (id, expose_id, is_active, source_name, source_id, url, creation_date, valid_until)
    VALUES 
"""

def insert_sources(df):
    print("Insert sources")
    before  = datetime.now()                         
    execute_query_many(connection, query_insert_sources, df.values.tolist())
    duration = (datetime.now() - before).total_seconds()                        
    print(f"{df.shape[0]} sources inserted in {duration} seconds.")

insert_sources(df_sources_new)

Insert sources
Query: 
    INSERT INTO live_ai_v2.sources (id, expose_id, is_active, source_name, source_id, url, creation_date, valid_until)
    VALUES 
('3884518475819788511','6400432849230923943',true,'is24','126782974','https://www.immobilienscout24.de/expose/126782974','2021-02-22T17:12:55.000Z','9999-12-31'),('-7862558759484577145','3402233425913400653',true,'immobilio','immobilioBMYJZS','https://www.immobilo.de/immobilien/penthouse-mit-aussicht-in-urbaner-lage-zeit-zum-geniessen-BMYJZS?utm_source=meta_searches&utm_medium=free&utm_id=033','2021-02-22T17:10:36.860Z','9999-12-31'),('-1339679137021876320','5910895991693760749',true,'is24','126782931','https://www.immobilienscout24.de/expose/126782931','2021-02-22T17:08:54.000Z','9999-12-31'),('621937499924091789','7849623999502979705',true,'is24','126782719','https://www.immobilienscout24.de/expose/126782719','2021-02-22T17:06:23.000Z','9999-12-31'),('4822154147887346806','5440182418862797720',true,'is24','126777608','https://www.im

In [45]:
df.sample(3)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
74,f51d9047034b249d8c516873f1535e51,Helle zwei Zimmer Dachgeschosswohnung in Köln-...,50827,180000.0,2.0,54.5,NaN,9.963303,3302.752294,543.0,NaN,NaN,Nordrhein-Westfalen,False,3.62,NaN,1970.0,ROOF_STOREY,NO_INFORMATION,NaN,False,NaN,NaN,False,False,True,False,2021-02-22T16:31:06.000Z,False,False,115.384615,NaN,"[{'id': '076e5fb2f8e4be5c6f2911e534f93b08', 'o...",False,False,0,2021-02-22T16:33:54.743Z,0,NaN,-163.30,-10.886667,-163.30,True,DE,DEU,place,neighbourhood,Köln,Ehrenfeld,Europe,Deutschland,de,European Union,50827.0,Nordrhein-Westfalen,NW,50.966872,6.905747,"Ossendorf, Köln, Nordrhein-Westfalen, Deutschland",1087863,-5,5,True,9.9,100221,67,0.000000,25,25.0,16.664,Ehrenfeld,182575.0,3350.0,3.689552,Köln,218000.0,4000,3.450000,NaN,156000.0,2862.385321,4.176923,NaN,50827.0,Köln,Ossendorf,50827,Köln,Ossendorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ossen

In [46]:
df['platforms.creationDate'].isnull().sum()

0

In [47]:
df_sources_new[df_sources_new['is_active'] == False]


,id,expose_id,is_active,source_name,source_id,url,creation_date,valid_until
75,8358157023821477052,-3932762746099402653,False,is24,126782174,https://www.immobilienscout24.de/expose/126782174,2021-02-22T16:31:06.000Z,2021-02-22T17:01:39.594Z
365,2272592014849099158,-2269239622289337774,False,is24,120035698,https://www.immobilienscout24.de/expose/120035698,2021-02-22T13:21:06.000Z,2021-02-22T17:01:16.381Z
366,2272592014849099158,-2269239622289337774,False,is24,120035698,https://www.immobilienscout24.de/expose/120035698,2021-02-22T13:21:06.000Z,2021-02-22T17:01:16.381Z
406,-3043475862740366270,1456040367573343036,False,is24,126778274,https://www.immobilienscout24.de/expose/126778274,2021-02-22T13:06:26.000Z,2021-02-22T17:01:32.005Z


In [48]:
df_sources_new.isnull().sum()

id               0
expose_id        0
is_active        0
source_name      0
source_id        0
url              0
creation_date    0
valid_until      0
dtype: int64

In [49]:
df[df['platforms.active'] == False]

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
75,f51d9047034b249d8c516873f1535e51,Helle zwei Zimmer Dachgeschosswohnung in Köln-...,50827,180000.0,2.0,54.50,NaN,9.963303,3302.752294,543.0,NaN,NaN,Nordrhein-Westfalen,False,3.62,NaN,1970.0,ROOF_STOREY,NO_INFORMATION,NaN,False,NaN,NaN,False,False,True,False,2021-02-22T16:31:06.000Z,False,False,115.384615,NaN,"[{'id': '076e5fb2f8e4be5c6f2911e534f93b08', 'o...",False,False,0,2021-02-22T16:33:54.743Z,0,NaN,-163.30,-10.886667,-163.30,True,DE,DEU,place,neighbourhood,Köln,Ehrenfeld,Europe,Deutschland,de,European Union,50827.0,Nordrhein-Westfalen,NW,50.966872,6.905747,"Ossendorf, Köln, Nordrhein-Westfalen, Deutschland",1087863,-5,5,True,9.9,100221,67,0.000,25,25.000,16.664,Ehrenfeld,182575.0,3350.0,3.689552,Köln,218000.0,4000,3.450000,NaN,156000.0,2862.385321,4.176923,NaN,50827.0,Köln,Ossendorf,50827,Köln,Ossendorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ossen

There are also price history entries existing where the entry is inactive!!!

# Create table price_history

In [51]:
def get_new_price_history(df):
    print("Create price history values")
    before  = datetime.now()
    
    # drop empty price histories
    df = df.drop(df[df['buyingPriceHistory.buyingPrice'].isnull()].index) 

    # create locations datframe
    columns = [
        "id",
        "expose_id",
        "purchase_price",
        "source_name",
        "creation_date",
    ] 
    #buyingPriceHistory.buyingPrice	buyingPriceHistory.platformName	buyingPriceHistory.creationDate	buyingPriceHistory	platforms.deactivationDate
    df_price_history_table = pd.DataFrame(columns=columns)
    df_price_history_table['id'] = get_price_history_id(df)
    df_price_history_table['expose_id'] = get_expose_id(df)
    df_price_history_table['purchase_price'] = df['buyingPriceHistory.buyingPrice'] 
    df_price_history_table['source_name'] = df['buyingPriceHistory.platformName']
    df_price_history_table['creation_date'] = df['buyingPriceHistory.creationDate']
    # df_price_history_table['creation_date'] = df['platforms.creationDate']
    # df_price_history_table['valid_until'] = np.where(df['platforms.deactivationDate'].notnull(),df['platforms.deactivationDate'],'9999-12-31')

    duration = (datetime.now() - before).total_seconds()                        
    print(f"Price history table created in {duration} seconds.")
    return df_price_history_table

df_price_history_new = get_new_price_history(df)
df_price_history_new = df_price_history_new.drop_duplicates('id')
df_price_history_new.head(3)

Create price history values
Price history table created in 0.045438 seconds.


,id,expose_id,purchase_price,source_name,creation_date
0,100080770576063210,6400432849230923943,129000.0,is24,2021-02-22T17:12:55.000Z
1,481568134793437289,3402233425913400653,535000.0,immobilio,2021-02-22T17:10:36.860Z
2,-3540035916177955922,5910895991693760749,403872.0,is24,2021-02-22T17:08:54.000Z


In [52]:
df[df['buyingPrice'] == 215000.0]

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
47,a98ec54cc8d4b6161628448b4647ef03,Diese Aussicht muss man genießen!,77886,215000.0,3.0,93.00,3.57,7.935484,2311.827957,738.0,NaN,NaN,Baden-Württemberg,False,4.12,NaN,1985.0,OTHER,FULLY_RENOVATED,NaN,False,NaN,NaN,True,False,True,False,2021-02-22T16:51:11.000Z,False,False,99.078341,NaN,"[{'id': '8dda08e936185642a5f1f02fe4dd52df', 'o...",False,False,0,2021-02-22T16:53:19.893Z,0,NaN,-150.30,-8.388837,-150.30,True,DE,DEU,postcode,postcode,NaN,NaN,Europe,Deutschland,de,European Union,77886.0,Baden-Württemberg,BW,48.647706,8.134519,"77886 Lauf, Deutschland",0,-5,5,False,4.1,0,35,18.658537,0,0.00,16.664,NaN,NaN,NaN,NaN,Lauf,199950.0,2150,4.241860,NaN,217000.0,2333.333333,4.081106,NaN,77886.0,Lauf,NaN,77886,Lauf,NaN,Ortenaukreis,Verwaltungsgemeinschaft Achern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lauf,NaN,NaN,NaN,NaN,NaN,Na

In [53]:
def get_source_id(df):
    return df.apply(lambda row: str(hash(get_expose_id_hash(row) + str(row['platforms.url']))), axis = 1)

def get_price_history_id(df):
    return df.apply(lambda row: str(hash(get_expose_id_hash(row) + str(row['platforms.name']))), axis = 1)

In [54]:
df_price_history_new[df_price_history_new[['expose_id','purchase_price','source_name']].duplicated(keep=False)]

,id,expose_id,purchase_price,source_name,creation_date
62,2512541500999153965,6446853054297504366,215000.0,is24,2021-02-22T16:41:28.000Z
63,2512541500999153965,6446853054297504366,215000.0,is24,2021-02-22T16:41:28.000Z
64,3511279484673814433,2687957956330067440,380000.0,is24,2021-02-22T16:40:23.000Z
65,3511279484673814433,2687957956330067440,380000.0,is24,2021-02-22T16:40:23.000Z
74,1437390219850129458,-3932762746099402653,180000.0,is24,2021-02-22T16:31:06.000Z
75,1437390219850129458,-3932762746099402653,180000.0,is24,2021-02-22T16:31:06.000Z
148,-4716916729834212476,-7647096259027067163,190000.0,ivd24,2021-02-22T15:41:53.643Z
149,-4716916729834212476,-7647096259027067163,190000.0,ivd24,2021-02-22T15:41:53.643Z
196,226263710173298895,1927383135424706066,249000.0,is24,2021-02-22T15:05:03.000Z
197,226263710173298895,1927383135424706066,249000.0,is24,2021-02-22T15:05:03.000Z


In [77]:
df_price_history_new.isnull().sum()

id                0
expose_id         0
purchase_price    0
source_name       0
creation_date     0
dtype: int64

In [50]:
query_insert_price_history = """
    INSERT INTO live_ai_v2.price_history (id, expose_id, purchase_price, source_name, creation_date)
    VALUES """

def insert_price_history(df):
    print("Insert sources")
    before  = datetime.now()                         
    execute_query_many(connection, query_insert_price_history, df.values.tolist())
    duration = (datetime.now() - before).total_seconds()                        
    print(f"{df.shape[0]} price history entries inserted in {duration} seconds.")

insert_price_history(df_price_history_new)

Insert sources
Query: 
    INSERT INTO live_ai_v2.price_history (id, expose_id, purchase_price, source_name, creation_date)
    VALUES ('2455437307175057218','7060630105729677006',129000.0,'is24','2021-02-22T17:12:55.000Z'),('-3495632460028748914','-7128812576709884680',535000.0,'immobilio','2021-02-22T17:10:36.860Z'),('-6028744366813829961','-59389727911579230',403872.0,'is24','2021-02-22T17:08:54.000Z'),('988559957019860321','4774022685237971713',259000.0,'is24','2021-02-22T17:06:23.000Z'),('-2032944097035100225','-7963511169867387098',3650000.0,'is24','2021-02-22T17:06:10.000Z'),('-8484887957819773619','-8655604143257177235',329000.0,'ivd24','2021-02-22T17:04:57.166Z'),('-2008014980804468945','-3396587632031484709',198000.0,'ebk','2021-02-22T17:04:12.318Z'),('-3009599155789875396','2909390151520569402',219000.0,'ebk','2021-02-22T17:04:04.604Z'),('5336508903248561027','8541750181749475446',855000.0,'ebk','2021-02-22T17:04:04.521Z'),('3970112649698084905','-2886712845400940349',469000

In [51]:
df[df['buyingPriceHistory.creationDate'].isnull()]

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
24,5d8291e03099ee788e461d6f3056031a,Ihr Eigentrum in guten Händen,60598,NaN,NaN,NaN,NaN,15.650000,NaN,NaN,NaN,NaN,Hessen,False,NaN,NaN,NaN,NO_INFORMATION,NO_INFORMATION,NaN,False,NaN,NaN,False,False,True,False,2021-02-22T17:02:15.508Z,False,False,NaN,NaN,"[{'id': '9c5982e460178520dce3dfddfe602440', 'o...",False,NaN,0,2021-02-22T17:02:16.316Z,0,NaN,0.0,0.0,0.0,True,DE,DEU,place,city,Frankfurt am Main,NaN,Europe,Deutschland,de,European Union,60598.0,Hessen,HE,50.110644,8.682092,"Frankfurt am Main, Hessen, Deutschland",763380,-5,5,True,7.3,66400,77,10.479452,25,25.00,16.664,NaN,NaN,NaN,NaN,Frankfurt am Main,0.0,5800,2.979310,NaN,0.0,8512.500000,2.206167,NaN,NaN,NaN,NaN,60598,Frankfurt am Main,NaN,NaN,NaN,NaN,NaN,60598.0,Frankfurt am Main,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [52]:
df[df['id'] == '1b5ebec0aeb005e866f94945fc0ef9c6']

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
623,1b5ebec0aeb005e866f94945fc0ef9c6,Eigentumswohnungen zu verkaufen - NEUBAU! Top ...,32105,NaN,3.0,60.0,NaN,10.25,NaN,615.0,NaN,NaN,Nordrhein-Westfalen,False,NaN,NaN,2021.0,NO_INFORMATION,FIRST_TIME_USE,NaN,False,NaN,NaN,True,False,True,False,2021-02-22T10:30:16.856Z,False,False,0.0,NaN,[],False,NaN,0,2021-02-22T13:54:04.551Z,0,NaN,0.0,0.0,0.0,False,DE,DEU,postcode,postcode,Kreis Lippe,NaN,Europe,Deutschland,de,European Union,32105.0,Nordrhein-Westfalen,NW,52.087916,8.746963,"32105 Bad Salzuflen, Deutschland",54254,-5,5,False,6.4,0,54,11.953125,0,25.0,16.664,Bad Salzuflen,96000.0,1600.0,3.9,Bad Salzuflen,96000.0,1600,4.875,NaN,202000.0,3366.666667,3.653465,NaN,NaN,NaN,NaN,32105,bad salzuflen,NaN,Kreis Lippe,NaN,Bad Salzuflen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bad Salzuflen,NaN,NaN,NaN,NaN,NaN,NaN,32105.0,"32105 Nordrhein-Westfalen, Bad Salzuflen","32105 Nordrh

## Create table exposes

## Fill table exposes

In [53]:
df.shape

(650, 157)

In [54]:
len(df['id'].unique())

600

In [55]:
df.head(2)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,oAddress.is24.district,originalAddress.postcode,originalAddress.location,originalAddress.district,address.county,address.municipality,address.town,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.borough,address.house_number,address.neighbourhood,address.office,address.road,address.suburb,originalAddress.street,address.allotments,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.locationQuery,oAddress.ebk.location,address.village,address.waterway,address.community_centre,address.road_type,address.restaurant,oAddress.ivd24.district,address.hamlet,address.building,address.commercial,address.bank,address.local_administrative_area,address.railway,address.fuel,address.locality,address.shop,address.landuse,address.residential,address.highway,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,address.school,oAddress.vrwohnen.street,oAddress.vrwohnen.postcode,oAddress.vrwohnen.location,address.industrial,address.doctors,address.boundary,address.quarter,address.emergency_service,address.cafe,address.car_sharing,address.farm,address.place,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory,platforms.deactivationDate
0,f5b7641a8b8198cab2bc469d5c1b8913,"Attraktive 1,5-Zimmer-Wohnung zwischen Mannhei...",69228,129000.0,1.0,48.00,4.44,9.791667,2687.500000,470.0,NaN,NaN,Baden-Württemberg,False,4.37,NaN,1984.0,APARTMENT,WELL_KEPT,NaN,False,NaN,1.0,True,False,True,False,2021-02-22T17:12:55.000Z,False,False,100.781250,NaN,"[{'id': '588ca284e7e9d0cfd67ac36c75492073', 'o...",False,False,0,2021-02-22T17:14:40.252Z,0,NaN,-76.58,-7.124031,-76.58,True,DE,DEU,place,neighbourhood,Mannheim,Friedrichsfeld,Europe,Deutschland,de,European Union,68229.0,Baden-Württemberg,BW,49.435667,8.572979,"68229 Mannheim, Deutschland",310658,-5,5,True,7.8,19417,76,9.807692,25,25.00,16.664,Friedrichsfeld,127200.0,2650.0,4.030189,Mannheim,148800.0,3100,3.870968,NaN,128000.0,2666.666667,4.406250,NaN,69228.0,Mannheim,Friedrichsfeld,69228,Mannheim,Friedrichsfeld,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

We have 600 unqiue exposes out of 650 rows. --> We have 50 duplicates (multiple platforms or price histories) 

1. New expose: Just insert expose —> create new **expose** entry, **source** entry and **price history** entry
   We will check the hashkey (expose_id) if expose already there

2. Existing expose: Price is the same 
    1. Different source: Same title and address is found —> create **price history**, create **source** entry
    2. Same source —> do nothing

3. Existing expose: Price is different —> **** **price history** entry is changed 
    1. Same source —> update the **price history**
        1. Price is higher than before —> check actual price from other sources and take the lowest for **exposes** entry ****
        2. Price is lower than before —>update latest price on **expose** entry
    2. Different source —> Create new **source** entry, create new **price history** entry
        1. Price is higher than before —> check actual price from other sources and take the lowest for **exposes** entry
        2. Price is lower than before —>update latest purchase_price on **expose**,
        
4. For all other exposes from the same source that are not there anymore: 
    1. Expose has only this source —> **Expose**, **price history**, **source** entries gets inactive 
    2. Expose has other sources 
        1. Has no more active sources  —> **Expose**, **price history**, **source** entries get inactive
        2. Has still active sources —> **price history**, **source** gets inactive
5. Reactivating inactive expose
    ID == title + zip (probably the price is different)
    1. Same source
        1. Price is different 
            1. price lower than before
            2. price higher than before
        2. Price is the same
    2. New source —> Expose, price history, source gets active again. and new entry for source and price

In [56]:
def get_new_exposes(df, existing_exposes):
    print("Create exposes values")
    before  = datetime.now()                         
    # drop duplicate locations (by zip)
    # df_temp = df.drop_duplicates('zip', keep='first') # Problem: drop city information
    # create locations datframe
    columns = [
        "id",
        "postal_code",
        "price_id",
        "source_id",
        "title",
        "purchase_price",
        "district",
        "street_number",
        "street",
        "address_display_name",
        "is_active",
        "property_type",
        "longitude",
        "latitude",
        "has_vaccancy",
        "is_fore_closure",
        "purchase_price_per_sqm",
        "commission_rate",
        "commission_total",
        "management_cost_given",
        "management_cost_estimated",
        "rent_price_per_sqm_given",
        "rent_price_per_sqm_estimated",
        "rent_price_total_estimated",
        "rent_price_total_given",
        "city_slug",
        "condition",
        "construction_year",
        "floors",
        "at_floor",
        "living_space",
        "rooms",
        "refurbishment_maintenance_reserve",
        "lift",
        "has_balcony",
        "is_private_offer",
        "has_garden",
        "image_captions",
        "image_urls",
        "cash_flow_gross",
        "cash_flow_net",
        "roi_equity",
        "mortgage_monthly",
        "mortgage_yearly",
        "mortgage_total",
        "interest_monthly",
        "interest_yearly",
        "interest_total",
        "personal_tax",
        "loan",
        "private_maintenance_reserve",
        "rent_price_index",
        "notary_rate",
        "real_estate_transfer_tax",
        "land_register_tax",
        "created_at",
        "updated_at"
    ] 
    df_exposes_table = pd.DataFrame(columns=columns)
    # hash as part of pandas
    df_exposes_table['id'] = get_expose_id(df)
    df_exposes_table['price_id'] = 0 # TODO add correct id
    df_exposes_table['source_id'] = np.nan # TODO add correct id
    df_exposes_table['postal_code'] = df['zip']
    df_exposes_table['title'] = df['title']
    # df_exposes_table['id'] = df.apply(lambda row: row['zip'] + row['title'], axis = 1)
    # df_exposes_table['id'] = get_hash(df_exposes_table['id'])
    # TODO fix: use our hash function
    df_exposes_table['purchase_price'] = df['buyingPrice']
    df_exposes_table['district'] = df['originalAddress.district'].fillna(df['address.city_district']).fillna(df['oAddress.is24.district']).fillna(df_locations['address.borough']).fillna(df['address.suburb'])
    df_exposes_table['street_number'] = df['address.house_number']   
    df_exposes_table['street'] = df['address.road'].fillna(df['originalAddress.street']).fillna(df['oAddress.is24.street'])   
    df_exposes_table['address_display_name'] = df['address.displayName']  
    df_exposes_table['is_active'] = True # TODO we only set it active it is really active. We need a check in here 
    df_exposes_table['property_type'] = df['apartmentType'] # TODO we either have it as FLAT, HOUSE or more granular 
    df_exposes_table["longitude"] = df['address.lon']
    df_exposes_table["latitude"] = df['address.lat']
    df_exposes_table['has_vaccancy'] = df['rented']
    df_exposes_table['is_fore_closure'] = df['foreClosure']
    df_exposes_table['is_leasehold'] = df['leasehold']
    df_exposes_table['purchase_price_per_sqm'] = df['buyingPrice'] / df['squareMeter']
    df_exposes_table['commission_rate'] = df['comission']
    df_exposes_table['commission_total'] = np.where(df['comission'].notnull(), df['comission'] * df['buyingPrice'] / 100, np.nan)
    df_exposes_table['management_cost_given'] = df['houseMoney']
    df_exposes_table['management_cost_estimated'] = np.nan # TODO calculate
    df_exposes_table['rent_price_total_estimated'] = np.nan # TODO calculate
    df_exposes_table['rent_price_total_given'] = df['rentPriceCurrent']
    df_exposes_table['rent_price_per_sqm_given'] = np.where(df['rentPriceCurrent'].notnull(), df['rentPriceCurrent'] / df['squareMeter'], np.nan)
    df_exposes_table['rent_price_per_sqm_estimated'] = None # TODO calculate
    df_exposes_table['city_slug'] = None # TODO
    df_exposes_table['condition'] = df['condition']
    df_exposes_table['construction_year'] = df['constructionYear']
    df_exposes_table['floors'] = df['numberOfFloors']
    df_exposes_table['at_floor'] = df['floor']
    df_exposes_table['living_space'] = df['squareMeter']
    df_exposes_table['rooms'] = df['rooms']
    df_exposes_table['refurbishment_maintenance_reserve'] = np.nan # TODO calculate
    df_exposes_table['has_lift'] = df['lift']
    df_exposes_table['has_balcony'] = df['balcony']
    df_exposes_table['is_private_offer'] = df['privateOffer']
    df_exposes_table['has_garden'] = df['garden']
    df_exposes_table['image_urls'] = df['images']  # TODO split json array into list of image urls
    df_exposes_table['image_captions'] = None # TODO split json array into list of images captions
    df_exposes_table['cash_flow_gross'] = np.nan # TODO calculate
    df_exposes_table['cash_flow_net'] = np.nan # TODO calculate
    df_exposes_table['roi_equity'] = np.nan # TODO calculate
    df_exposes_table['mortgage_monthly'] = np.nan # TODO calculate
    df_exposes_table['mortgage_yearly'] = np.nan # TODO calculate
    df_exposes_table['mortgage_total'] = np.nan # TODO calculate
    df_exposes_table['interest_monthly'] = np.nan # TODO calculate
    df_exposes_table['interest_yearly'] = np.nan # TODO calculate
    df_exposes_table['interest_total'] = np.nan # TODO calculate
    df_exposes_table['personal_tax'] = np.nan # TODO calculate
    df_exposes_table['loan'] = np.nan # TODO calculate
    df_exposes_table['private_maintenance_reserve'] =  np.nan # TODO calculate
    df_exposes_table['rent_price_index_given'] =  np.nan # TODO calculate
    df_exposes_table['rent_price_index_estimated'] = np.nan # TODO calculate
    df_exposes_table['notary_rate'] = np.nan # TODO calculate
    df_exposes_table['real_estate_transfer_tax'] = np.nan # TODO calculate
    df_exposes_table['land_register_tax'] = np.nan # TODO calculate

    result = df_exposes_table[~df_exposes_table['id'].isin(existing_exposes)]
    duration = (datetime.now() - before).total_seconds()                        
    print(f"Locations table created in {duration} seconds.")
    return result

# TODO bring back duplicate exposes
# for the purpose of execising we will drop duplicate exposes
df_without_duplicates = df.drop_duplicates('id')
existing_exposes = []
df_exposes_new = get_new_exposes(df_without_duplicates,existing_exposes)
# TODO create mechanism to deal with duplicates
df_exposes_new = df_exposes_new.drop_duplicates('id')
df_exposes_new.head(3)

Create exposes values
Locations table created in 0.057393 seconds.


,id,postal_code,price_id,source_id,title,purchase_price,district,street_number,street,address_display_name,is_active,property_type,longitude,latitude,has_vaccancy,is_fore_closure,purchase_price_per_sqm,commission_rate,commission_total,management_cost_given,management_cost_estimated,rent_price_per_sqm_given,rent_price_per_sqm_estimated,rent_price_total_estimated,rent_price_total_given,city_slug,condition,construction_year,floors,at_floor,living_space,rooms,refurbishment_maintenance_reserve,lift,has_balcony,is_private_offer,has_garden,image_captions,image_urls,cash_flow_gross,cash_flow_net,roi_equity,mortgage_monthly,mortgage_yearly,mortgage_total,interest_monthly,interest_yearly,interest_total,personal_tax,loan,private_maintenance_reserve,rent_price_index,notary_rate,real_estate_transfer_tax,land_register_tax,created_at,updated_at,is_leasehold,has_lift,rent_price_index_given,rent_price_index_estimated
0,7060630105729677006,69228,0,NaN,"Attraktive 1,5-Zimmer-Wohnung zwischen Mannhei...",129000.0,Friedrichsfeld,NaN,NaN,"68229 Mannheim, Deutschland",True,APARTMENT,8.572979,49.435667,False,False,2687.500000,4.44,5727.6,NaN,NaN,NaN,None,NaN,NaN,None,WELL_KEPT,1984.0,1.0,NaN,48.00,1.0,NaN,NaN,True,False,False,None,"[{'id': '588ca284e7e9d0cfd67ac36c75492073', 'o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN
1,-7128812576709884680,71083,0,NaN,Penthouse mit Aussicht in urbaner Lage. Zeit z...,535000.0,NaN,NaN,NaN,"71083 Herrenberg, Deutschland",True,PENTHOUSE,8.867472,48.595557,False,False,5522.295623,4.44,23754.0,NaN,NaN,NaN,None,NaN,NaN,None,FIRST_TIME_USE,2021.0,4.0,2.0,96.88,3.0,NaN,NaN,False,False,False,None,"[{'id': '86919a5116d0de3adc8cc2d5e5f06b59', 'o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN
2,-59389727911579230,12049,0,NaN,Gewerbefläche in 1A-Lage mit 3 Räumen,403872.0,Neukölln (Ortsteil),51,Leinestraße,"Netzwerk behinderter Frauen Berlin e.V., Leine...",True,APARTMENT,13.426049,52.473055,True,False,4200.000000,NaN,NaN,NaN,NaN,0.528754,None,NaN,50.845,None,NO_INFORMATION,1910.0,NaN,NaN,96.16,NaN,NaN,NaN,True,False,False,None,"[{'id': 'd6a168d06fd728f07ddd8eec06222ff9', 'o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN


In [57]:
df_exposes_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 597 entries, 0 to 649
Data columns (total 61 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 597 non-null    object 
 1   postal_code                        597 non-null    object 
 2   price_id                           597 non-null    int64  
 3   source_id                          0 non-null      float64
 4   title                              597 non-null    object 
 5   purchase_price                     583 non-null    float64
 6   district                           384 non-null    object 
 7   street_number                      94 non-null     object 
 8   street                             192 non-null    object 
 9   address_display_name               597 non-null    object 
 10  is_active                          597 non-null    bool   
 11  property_type                      597 non-null    object 

In [58]:
df_exposes_new[df_exposes_new['id'] == '-8107215910107044932']

,id,postal_code,price_id,source_id,title,purchase_price,district,street_number,street,address_display_name,is_active,property_type,longitude,latitude,has_vaccancy,is_fore_closure,purchase_price_per_sqm,commission_rate,commission_total,management_cost_given,management_cost_estimated,rent_price_per_sqm_given,rent_price_per_sqm_estimated,rent_price_total_estimated,rent_price_total_given,city_slug,condition,construction_year,floors,at_floor,living_space,rooms,refurbishment_maintenance_reserve,lift,has_balcony,is_private_offer,has_garden,image_captions,image_urls,cash_flow_gross,cash_flow_net,roi_equity,mortgage_monthly,mortgage_yearly,mortgage_total,interest_monthly,interest_yearly,interest_total,personal_tax,loan,private_maintenance_reserve,rent_price_index,notary_rate,real_estate_transfer_tax,land_register_tax,created_at,updated_at,is_leasehold,has_lift,rent_price_index_given,rent_price_index_estimated


In [59]:
df_exposes_new[df_exposes_new['id'] == '46508268396587403']

,id,postal_code,price_id,source_id,title,purchase_price,district,street_number,street,address_display_name,is_active,property_type,longitude,latitude,has_vaccancy,is_fore_closure,purchase_price_per_sqm,commission_rate,commission_total,management_cost_given,management_cost_estimated,rent_price_per_sqm_given,rent_price_per_sqm_estimated,rent_price_total_estimated,rent_price_total_given,city_slug,condition,construction_year,floors,at_floor,living_space,rooms,refurbishment_maintenance_reserve,lift,has_balcony,is_private_offer,has_garden,image_captions,image_urls,cash_flow_gross,cash_flow_net,roi_equity,mortgage_monthly,mortgage_yearly,mortgage_total,interest_monthly,interest_yearly,interest_total,personal_tax,loan,private_maintenance_reserve,rent_price_index,notary_rate,real_estate_transfer_tax,land_register_tax,created_at,updated_at,is_leasehold,has_lift,rent_price_index_given,rent_price_index_estimated


In [60]:
len(df_exposes_new.columns)

61

In [61]:
# query_insert_exposes = """
#     INSERT INTO live_ai_v2.exposes (id, postal_code, price_id, source_id, title, purchase_price, district, street_number, street, address_display_name, zip_code, is_active, property_type, longitude, latitude, has_vaccancy, is_fore_closure, purchase_price_per_sqm, commission_rate, commission_total, management_cost_given, management_cost_estimated, rent_price_per_sqm_given, rent_price_per_sqm_estimated, rent_price_total_estimated, rent_price_total_given, city_slug, condition, construction_year, floors, at_floor, living_space, rooms, refurbishment_maintenance_reserve, lift, has_balcony, is_private_offer, has_garden, image_captions, image_urls, cash_flow_gross, cash_flow_net, roi_equity, mortgage_monthly, mortgage_yearly, mortgage_total, interest_monthly, interest_yearly, interest_total, personal_tax, loan, private_maintenance_reserve, rent_price_index, notary_rate, real_estate_transfer_tax, land_register_tax)
#     VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
# """
query_insert_exposes = """
    INSERT INTO live_ai_v2.exposes (id,postal_code,price_id)
    VALUES """

def insert_exposes(df):
    print("Insert exposes")
    before  = datetime.now()                         
    execute_query_many(connection, query_insert_exposes, df[['id','postal_code','price_id']].values.tolist())
    duration = (datetime.now() - before).total_seconds()                        
    print(f"{df.shape[0]} exposes inserted in {duration} seconds.")

# TODO bring back exposes
insert_exposes(df_exposes_new)

Insert exposes
Query: 
    INSERT INTO live_ai_v2.exposes (id,postal_code,price_id)
    VALUES ('7060630105729677006','69228',0),('-7128812576709884680','71083',0),('-59389727911579230','12049',0),('4774022685237971713','52074',0),('-7963511169867387098','60323',0),('-8655604143257177235','93047',0),('-3396587632031484709','67071',0),('2909390151520569402','12169',0),('8541750181749475446','15526',0),('-2886712845400940349','31141',0),('2947346139345756536','92690',0),('3313861057574232735','09119',0),('-766298807039107006','76831',0),('2910463986246701858','67547',0),('2813994498144452055','67547',0),('-473035608153973228','67547',0),('-9149404186383799707','67547',0),('3940952160933418518','67547',0),('-6372207065386585280','10969',0),('-832935503423179812','10117',0),('2603627706629988399','61350',0),('4209612892898437381','58097',0),('7275233390671249185','66606',0),('-6619907509030019058','45468',0),('-717693912853636865','60598',0),('8770122436106623124','60306',0),('-56113252482

How to insert data among multiple tables?   
1. Create dataframe by dataframe --> exposes, sources, history     
2. For loop for every expose 
